<a href="https://colab.research.google.com/github/Jamamaboy/001/blob/main/superai_lv1_hack1_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os
from transformers import BertTokenizer, BertForTokenClassification, Trainer, TrainingArguments
from transformers import DataCollatorForTokenClassification
from datasets import load_dataset, load_metric

In [2]:
# Upload
from google.colab import files

# อัปโหลดไฟล์จากเครื่องของคุณ
uploaded = files.upload()

# แสดงชื่อไฟล์ที่อัปโหลด
for file_name in uploaded.keys():
    print(f"Uploaded file: {file_name}")

Saving Super AI Named Entity Recognition.zip to Super AI Named Entity Recognition.zip
Uploaded file: Super AI Named Entity Recognition.zip


Super AI Named Entity Recognition.zip


In [4]:
dataset_extracted_path = "./extracted_files"
!unzip -o "{file_name}" -d "{dataset_extracted_path}"

Archive:  Super AI Named Entity Recognition.zip
  inflating: ./extracted_files/eval/eval/04278.txt  
  inflating: ./extracted_files/eval/eval/04279.txt  
  inflating: ./extracted_files/eval/eval/04280.txt  
  inflating: ./extracted_files/eval/eval/04281.txt  
  inflating: ./extracted_files/eval/eval/04282.txt  
  inflating: ./extracted_files/eval/eval/04283.txt  
  inflating: ./extracted_files/eval/eval/04284.txt  
  inflating: ./extracted_files/eval/eval/04285.txt  
  inflating: ./extracted_files/eval/eval/04286.txt  
  inflating: ./extracted_files/eval/eval/04287.txt  
  inflating: ./extracted_files/eval/eval/04288.txt  
  inflating: ./extracted_files/eval/eval/04289.txt  
  inflating: ./extracted_files/eval/eval/04290.txt  
  inflating: ./extracted_files/eval/eval/04291.txt  
  inflating: ./extracted_files/eval/eval/04292.txt  
  inflating: ./extracted_files/eval/eval/04293.txt  
  inflating: ./extracted_files/eval/eval/04294.txt  
  inflating: ./extracted_files/eval/eval/04295.txt 

In [6]:
tag_list = pd.read_csv('./extracted_files/tag_list.csv')
tag2id = {row['tag']: row['class'] for _, row in tag_list.iterrows()}
id2tag = {v: k for k, v in tag2id.items()}

In [8]:
print(tag2id)

{'O': 0, 'B_ORG': 1, 'B_PER': 2, 'B_LOC': 3, 'B_MEA': 4, 'I_DTM': 5, 'I_ORG': 6, 'E_ORG': 7, 'I_PER': 8, 'B_TTL': 9, 'E_PER': 10, 'B_DES': 11, 'E_LOC': 12, 'B_DTM': 13, 'B_NUM': 14, 'I_MEA': 15, 'E_DTM': 16, 'E_MEA': 17, 'I_LOC': 18, 'I_DES': 19, 'E_DES': 20, 'I_NUM': 21, 'E_NUM': 22, 'B_TRM': 23, 'B_BRN': 24, 'I_TRM': 25, 'E_TRM': 26, 'I_TTL': 27, 'I_BRN': 28, 'E_BRN': 29, 'E_TTL': 30, 'B_NAME': 31}


In [9]:
print(id2tag)

{0: 'O', 1: 'B_ORG', 2: 'B_PER', 3: 'B_LOC', 4: 'B_MEA', 5: 'I_DTM', 6: 'I_ORG', 7: 'E_ORG', 8: 'I_PER', 9: 'B_TTL', 10: 'E_PER', 11: 'B_DES', 12: 'E_LOC', 13: 'B_DTM', 14: 'B_NUM', 15: 'I_MEA', 16: 'E_DTM', 17: 'E_MEA', 18: 'I_LOC', 19: 'I_DES', 20: 'E_DES', 21: 'I_NUM', 22: 'E_NUM', 23: 'B_TRM', 24: 'B_BRN', 25: 'I_TRM', 26: 'E_TRM', 27: 'I_TTL', 28: 'I_BRN', 29: 'E_BRN', 30: 'E_TTL', 31: 'B_NAME'}


In [11]:
def read_data(path):
    data = []
    for file in os.listdir(path):
        with open(os.path.join(path,file),'r',encoding='utf-8') as f:
            text = f.read()
            data.append({'text':text}) # You might need to adjust this based on your data format
    return data

In [22]:
def read_data_to_dataframe(path):
    all_data = []
    for file in os.listdir(path):
        file_path = os.path.join(path, file)
        if os.path.isfile(file_path):
            with open(file_path, 'r', encoding='utf-8') as f:
                for line in f:
                    if line.strip():
                        parts = line.strip().split('\t')
                        if len(parts) == 4:  # เช็คว่ามี 4 คอลัมน์
                            token, pos, bio, cls = parts
                            all_data.append({"Token": token, "POS": pos, "BIO": bio, "CLS": cls})
    return pd.DataFrame(all_data)

# เรียกใช้ฟังก์ชันกับโฟลเดอร์ที่ต้องการ
train_df = read_data_to_dataframe('./extracted_files/train/train/')
eval_df = read_data_to_dataframe('./extracted_files/eval/eval/')

In [23]:
train_df

,Token,POS,BIO,CLS
0,ทีม,NN,B_ORG,B_CLS
1,ไทย,NN,E_ORG,I_CLS
2,เชิญ,VV,O,I_CLS
3,สื่อ,NN,O,I_CLS
4,ต่างชาติ,AJ,O,I_CLS
...,...,...,...,...
2804216,ไม่,NG,O,I_CLS
2804217,มี,VV,O,I_CLS
2804218,อะไร,PR,O,I_CLS
2804219,พิเศษ,VV,O,I_CLS


In [24]:
eval_df

,Token,POS,BIO,CLS
0,สาย,NN,B_ORG,B_CLS
1,การ,NN,I_ORG,I_CLS
2,บิน,NN,I_ORG,I_CLS
3,แห่ง,NN,I_ORG,I_CLS
4,ชาติ,NN,I_ORG,I_CLS
...,...,...,...,...
248465,ขยาย,VV,O,I_CLS
248466,ผล,NN,O,I_CLS
248467,การ,FX,O,I_CLS
248468,จับกุม,VV,O,I_CLS


In [27]:
def prepare_dataset(data):
    tokens = []
    labels = []
    for sentence in data:
        tokens.append([token for token, label in sentence])
        labels.append([label for token, label in sentence])
    return Dataset.from_dict({"tokens": tokens, "labels": labels})


In [28]:
train_dataset = prepare_dataset(train_df)
eval_dataset = prepare_dataset(eval_df)

print(train_dataset)

ValueError: not enough values to unpack (expected 2, got 1)